### Develop code to simulate pairwise competition experiments under the M3 model of population dynamics

In [ ]:
     
import numpy as np

import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
### Update dependent parameters according to input
import os
import os.path
from os import path

## create export directory if necessary
## foldernames for output plots/lists produced in this notebook
import os
FIG_DIR = f'./figures/develop_pairwise/'
os.makedirs(FIG_DIR, exist_ok=True)
print("All  plots will be stored in: \n" + FIG_DIR)

In [ ]:


### execute script to load modules here
exec(open('setup_aesthetics.py').read())

In [ ]:
DATASET_COLOR = 'darkorange'


In [ ]:
## cell flagged with tag parameters
# constrict to subset for debugging purposes
# setting this to None makes all curves
NO_CURVES = 20

DIST = 'all_traits_vary'
INITIAL_FREQ = 0.5

In [ ]:
SUFFIX_DATASET = 'warringer2003/'

OUTPUT_DIR_DATASET = './output/' + SUFFIX_DATASET
os.makedirs(OUTPUT_DIR_DATASET, exist_ok=True)

### Load wildtype data

In [ ]:
## setup background files from PLATEAU FINDER
exec(open('setup_plateau_finder_warringer2003.py').read())


In [ ]:
INDEX_COL = [0,1,2,3,4]
list_na_representations = ['not_present', 'failed_to_compute']

In [ ]:
PCWS_TRAITS_WARRINGER = './output/df_M3_traits.csv'
df_warringer = pd.read_csv(PCWS_TRAITS_WARRINGER, header = 0, index_col= INDEX_COL,\
                                  float_precision=None, na_values=list_na_representations)


In [ ]:
### define default wild_type
df_wildtypes = df_warringer[df_warringer['is_wildtype']==True]

WILDTYPE = df_wildtypes.median(axis = 0)

### Load mutant data (averaged)

In [ ]:

PCWS_TRAITS_WARRINGER_AVERAGED = './output/df_M3_traits_averaged.csv'
df_averaged = pd.read_csv(PCWS_TRAITS_WARRINGER_AVERAGED, header = 0, float_precision=None)

In [ ]:
### assign wild-type label
def is_wildtype(row):
    genotype = row['genotype']
    
    if genotype == 'BY4741':
        return True
    else:
        return False
    

row = df_averaged.iloc[0]
is_wildtype(row)

In [ ]:
df_averaged['is_wildtype'] = df_averaged.apply(is_wildtype, axis =1)

In [ ]:
### append mutant values (averaged) to set of individual wild-type strains
df_knockouts = df_averaged[~df_averaged['is_wildtype']]
df_knockouts = df_knockouts
df_input = df_wildtypes.reset_index().append(df_knockouts.reset_index())

In [ ]:
### restore index
index_col_names = df_warringer.index.names
df_input = df_input.set_index(index_col_names)


### Load trait data into the standard form required by Michaels code

In [ ]:
n_knockouts = df_knockouts.shape[0]

In [ ]:
### growth rates
gs = np.zeros(n_knockouts+1)
gs[0] = WILDTYPE['gmax']
gs[1:] = df_knockouts['gmax'].values

### lag times
ls = np.zeros(n_knockouts+1)
ls[0] = WILDTYPE['lag']
ls[1:] = df_knockouts['lag'].values

### yield
Ys = np.zeros(n_knockouts+1)
Ys[0] = WILDTYPE['yield']
Ys[1:] = df_knockouts['yield'].values


### Define initial condition for pairwise growth cycle

In [ ]:
## set initial proportion of the single mutant strain
x0 = 0.001 


In [ ]:
### set initial resource concentrations

CONCENTRATION_GLUCOSE = 20/180 * 1e3 # concentrations are recored  in milliMolar, to match the units of yield
print(CONCENTRATION_GLUCOSE)

In [ ]:
### define default initial_OD
OD_START = 0.05  #df_warringer['od_start'].median()

### compare to initial OD in the monoculture cycles
fig, ax = plt.subplots(figsize = (FIGWIDTH_TRIPLET, FIGHEIGHT_TRIPLET))

ax = df_warringer['od_start'].hist(bins=41, color = DATASET_COLOR, alpha = 0.6, log = True, rasterized = True)


ax.axvline(OD_START, color = 'tab:red', label = f'median value: $N_0={OD_START:.3f}$')
ax.legend()
ax.set_xlabel('initial OD')
ax.set_ylabel('no. growth curves')

### Convert the biomass yield to the effective yield $\nu$ 

In [ ]:
from bulk_simulation_code import CalcRelativeYield

In [ ]:
nus = CalcRelativeYield(Ys=Ys, R0 = CONCENTRATION_GLUCOSE, N0 = OD_START)

### Calculate vector of saturation times using the results from the M3 model

In [ ]:
from m3_model import CalcRelativeSaturationTime as CalcSaturationTimeExact
from m3_model import CalcFoldChange

In [ ]:
%%time

g1, l1, nu1 = gs[0], ls[0], nus[0] # get traits of the resident(wild-type) 
x1, x2 = 1-x0,x0                   # set initial frequency of the two strains

xs_final = np.zeros_like(gs)       # prepare data container

for i in range(len(gs)):
    g2, l2, nu2 = gs[i], ls[i], nus[i] # get traits of the invader


    
    # compute biomass fold-change
    tsat = CalcSaturationTimeExact(xs=[x1,x2], gs = [g1,g2], ls= [l1,l2], nus = [nu1,nu2] )
    fcs1, fcs2 = CalcFoldChange(t = tsat, g = [g1,g2], l = [l1,l2])
    
    # compute final frequency
    xs_final[i] = x2*fcs2/(x1 *fcs1 + x2*fcs2)
    

### Calculate pairwise selection coefficient under different encodings


In [ ]:
from bulk_simulation_code import CalcTotalSelectionCoefficientLogit

In [ ]:
si_wt_logit = CalcTotalSelectionCoefficientLogit(xs = np.ones_like(xs_final)*x0, xs_final = xs_final)

### Plot distribution of fitness effects

In [ ]:
fig, ax = plt.subplots()
_ = ax.hist(si_wt_logit, bins = 41)
ax.set_xlabel("pairwise selection coefficient in co-culture")
ax.set_ylabel('count')